In [2]:
from docplex.mp.model import Model
import pandas as pd
import numpy as np
import random
import copy
import time
import os
import math
import scipy.optimize
from scipy.spatial import distance

In [3]:
# %%javascript
# Jupyter.notebook.execute_cells([0])
# from IPython.display import Javascript
# Javascript("Jupyter.notebook.execute_cells([6])")

In [101]:
#### Scatter Search Initialization ------------------- Global Variables 

P_Size =100      # Size of Subset 
b = 10
b1 = 5
b2 = 5
Max_Iter = 3

In [201]:
## FOR LOOOPP FOR 15 times 
for j_for in range(0,15):
    print('The %d iteration \n'%(j_for))
    ########## THE SCATTER SEARCH ALGORITHM  - LOWER LEVEL COMPARISON ######################
    start = time.time()
    # ----------------------------------------------- STEP  1,2,3 -----------------------------------------------------
    PRICE_DIVERSE_SET, freq_matrix = diversification_generation(K)
    PRICE_IMRPOVE_SET = improvement_method(PRICE_DIVERSE_SET) 
    mid = time.time()
    # print('Time for Improvement method' , mid - start)

    Ref_Set1 , P_Set =  high_quality_subset(PRICE_IMRPOVE_SET )         #Storing the References Sets 
    Ref_Set = copy.deepcopy( Ref_Set1 )
    Ref_Set_Final, Ref_Set2 = diverse_subset(P_Set, Ref_Set  ) 

    for i in range(0,Max_Iter):                                                    #Outer Most Loop , Iterator 

    #     print('RefSetFinal', Ref_Set_Final, '\n\nRefSet1:', Ref_Set1 , '\n\nRefSet1:' , Ref_Set2)   
        New_Elements = True 
        while_it = 0
        while New_Elements:                                                # If RefSet1 or RefSet2 is changed do it , else stop 
            while_it += 1 
#             print('Number of Times in while iteration', while_it)
    #         Max_Subsets_2 =  ncr(b,2)
            New_Elements = False 
            for subset in subsets:                                        # Combining solutions for each subset in the Ref Set
    #             print('Set1 \n',Ref_Set1_F(Ref_Set1))                                      # Print olf ref_set1 and ref_Set2 
    #             print('Set2 \n',Ref_Set2_F(Ref_Set2)) 
    #             print(subset[0], subset[1])
    # ----------------------------------------------- 5 -----------------------------------------------------
                combined_solutions = solution_combination( Ref_Set_Final[subset[0]], Ref_Set_Final[subset[1]])  #Gives 3 comb solns
    #             print('Improving the combined solutions ')
                imp_combined_solutions   = improvement_method(combined_solutions)     # IMPROVING THE SOLUTIONS COMBINATIONS 
    #             CHECK if the combined_soln is there in the Reference SET ? 
                for combined_sol in imp_combined_solutions:                                       # for each comb solnt
                    combined_sol_unique =   combined_sol_unique_f(combined_sol, Ref_Set_Final)    # checking if it's unique 
    #                 print('Combined soln is unique ? \n', combined_sol_unique )
                # IS COMBINED SOL BETTER OR DIVERSE  ? 
                    if combined_sol_unique:        
    #                     obj_solver = master_lower_level(K, J ,  combined_sol , R_K, BUDGETS, S_K  )   # CPLEX - Getting Obj of comb sol
                        obj_solver = lower_level_optimal(K,J, combined_sol , BUDGETS , R_K, S_K )  # HEURSITIC 
                        upper_obj_val = obj_solver[2]
                        worst_obj_s =  worst_objf(Ref_Set1)
                        worst_obj_upper_val = worst_obj_s[0]
                        worst_obj_lower_val = worst_obj_s[1]
                        if upper_obj_val >=   worst_obj_upper_val:                       #New solution has a better obj function of leader ? 
    #                     print(upper_obj_val)
    #  IF I am comparing lower level obj , it must be compared with the worst_upper_level obj solution 
                            if  upper_obj_val ==   worst_obj_upper_val  :
    #                             print('Upper Level Objective of combined and worst in ref set are equal:', upper_obj_val ,  worst_obj_upper_val)
                                if obj_solver[1] > worst_obj_lower_val:
    #                                 print('Lower Level Objective of combined is better:', obj_solver[1]  ,  worst_obj_lower_val)
                                    obj_improve_count = 0
    #                                 print('Found a Better Obj Sol with lower level obj')
                                    nest_comb_soln = [0 for x in range(3)]      # Perhaps it's better to have a structure of 3 empty space
                                    nest_comb_soln[0] = combined_sol  
                                    nest_comb_soln[1] = upper_obj_val 
                                    Ref_Set1 = sorted(Ref_Set1, key=lambda x: x[1], reverse=True)
                                    Ref_Set1.pop(-1)                                    # Should be arranged to pop last element 
                                    Ref_Set1.append(nest_comb_soln)
                #                 Updating Ref Set Final 
                                    Ref_Set_Final = copy.deepcopy( Ref_Set1 )
                                    Ref_Set_Final.extend(Ref_Set2)
                                    New_Elements = True 
                            else:
                                obj_improve_count = 0
            #                         print('Found a Better Obj Sol')
                                nest_comb_soln = [0 for x in range(3)]      # Perhaps it's better to have a structure of 3 empty space
                                nest_comb_soln[0] = combined_sol  
                                nest_comb_soln[1] = upper_obj_val 
                                Ref_Set1 = sorted(Ref_Set1, key=lambda x: x[1], reverse=True)
                                Ref_Set1.pop(-1)                                    # Should be arranged to pop last element 
                                Ref_Set1.append(nest_comb_soln)
            #                 Updating Ref Set Final 
                                Ref_Set_Final = copy.deepcopy( Ref_Set1 )
                                Ref_Set_Final.extend(Ref_Set2)
                                New_Elements = True 

        if i < Max_Iter :
    #          WORKS FINE 
#             print('i Iteration',i)
    #         print('Set1 \n',Ref_Set1_F(Ref_Set1))                                      # Print old ref_set1 and ref_Set2 
    #         print('Set2 \n',Ref_Set2_F(Ref_Set2) ,'\n\n', Ref_Set_Final)                                           
            PRICE_DIVERSE_SET, freq_matrix = diversification_generation(K)
            Pop_Set =   [[0 for x in range(3)] for y in range( P_Size )] 
            for i in range(0, len(Pop_Set)):
                Pop_Set[i][0] = PRICE_DIVERSE_SET[i]
            Ref_Set = copy.deepcopy( Ref_Set1 )
            Ref_Set_Final, Ref_Set2 = diverse_subset(Pop_Set, Ref_Set ) 
    #         print('Set1 \n',Ref_Set1_F(Ref_Set1))                                      # Print old ref_set1 and ref_Set2 
    #         print('Set2 \n',Ref_Set2_F(Ref_Set2),'\n\n', Ref_Set_Final) 

    end = time.time()
    print('Time Elapsed' , end - start)
    print(instance_name)

    ## PRINTING THE SOLUTION   -- Heuristic    #####################

    Opt_Prices = sorted(Ref_Set1, key=lambda x: x[1], reverse=True )[0][0]
    Cust_Assign =  lower_level_optimal(K,J, Opt_Prices , BUDGETS , R_K, S_K )
    print('Optimal Prices are ', Opt_Prices)
    print('Objective of Leader and Follower are',  Cust_Assign[2], Cust_Assign[1]  )
    print('Assignment of Customers ',  Cust_Assign[0] )

    ########  Adding Rows to dataframe  ############
    new_row = { 'leader_obj':Cust_Assign[2], 'follower_obj':Cust_Assign[1], 'CPU_time':end - start}
    df_result = df_result.append(new_row, ignore_index=True)

The 0 iteration 

Time Elapsed 478.0809164047241
test_90k_20ta_10.txt
Optimal Prices are  [25.0, 24.0, 27.0, 25.0, 18.0, 30.0, 26.0, 23, 22.0, 25.0, 27.0, 28, 25, 26.0, 29.0, 25, 28, 28.0, 21.0, 33.0, 24.0, 24.0, 25.0, 22.0, 16.0, 25.0, 27.0, 26.0, 20.0, 30.0, 14.0, 18.0, 21.0, 16.0, 29.0, 21.0, 14.0, 22.0, 14.0, 22.0, 25.0, 32.0, 19.0, 14.0, 24.0, 11.0, 27.0, 19.0, 12.0, 25.0, 27.0, 26.0, 25.0, 21.0, 9.0, 14.0, 26.0, 19.0, 21.0, 24.0, 13.0, 17.0, 19.0, 23.0, 30.0, 15.0, 16.0, 21.0, 32.0, 22.0, 23.0, 21.0, 19.0, 17.0, 13.0, 9.0, 9.0, 20.0, 17.0, 14.0, 16.0, 17.0, 20.0, 29.0, 29.0, 9.0, 14.0, 22.0, 27.0, 13.0]
Objective of Leader and Follower are 1900.0 1084.7999999999997
Assignment of Customers  [19, 5, 5, 5, 5, 9, 7, 7, 8, 5, 5, 8, 2, 19, 5, 9, 19, 19, 7, 19, 13, 2, 5, 7, 9, 8, 8, 19, 14, 5, 5, 5, 3, 0, 5, 5, 13, 5, 5, 5, 9, 8, 2, 5, 13, 8, 3, 7, 9, 19, 19, 2, 9, 5, 5, 5, 9, 19, 14, 2, 5, 19, 5, 5, 0, 2, 5, 12, 4, 18, 5, 7, 19, 5, 2, 9, 19, 8, 10, 4, 8, 19, 14, 10, 19, 1, 8, 9, 0, 19]

Time Elapsed 168.63459992408752
test_90k_20ta_10.txt
Optimal Prices are  [24, 20, 22, 21, 26, 26, 22, 25, 34, 31, 18, 24, 22, 25, 23, 25, 22, 24, 24, 28, 22, 27, 28.0, 24, 23, 25, 19, 26, 28, 11, 18, 18, 17, 27, 12, 31, 21, 20, 22, 26, 21, 18, 31, 15, 19, 10, 23, 18.0, 19, 20, 19, 25, 21, 30, 22, 27, 16, 26, 18, 19, 21, 28, 21, 20, 29.0, 28, 21, 32, 28, 23, 28, 19, 33, 27, 29, 18, 26, 27, 27, 19, 27, 18.0, 21, 21, 26, 28, 25, 23.0, 20, 27]
Objective of Leader and Follower are 1844 1149.0
Assignment of Customers  [4, 11, 11, 7, 2, 3, 18, 7, 8, 18, 11, 2, 2, 4, 2, 7, 7, 19, 15, 13, 18, 2, 2, 7, 2, 8, 12, 13, 14, 11, 11, 5, 3, 0, 11, 3, 13, 11, 2, 11, 2, 1, 2, 0, 13, 7, 3, 7, 17, 4, 2, 2, 12, 11, 3, 11, 15, 7, 7, 11, 11, 2, 2, 2, 11, 1, 11, 12, 4, 17, 11, 7, 19, 2, 2, 13, 2, 11, 7, 4, 13, 4, 14, 2, 11, 1, 8, 7, 0, 4]
The 10 iteration 

Time Elapsed 165.31121158599854
test_90k_20ta_10.txt
Optimal Prices are  [29, 23, 16, 31, 23, 25, 28, 24, 36, 21, 36, 33, 23, 24, 26, 23, 23, 19.0, 20, 27,

In [202]:
#   Printing the dataframe 
print('\033[1m', instance_name , '\033[0m')
df_result

 test_90k_20ta_10.txt 


,leader_obj,follower_obj,CPU_time
0,1900.0,1084.8,478.080916
1,1849.0,1138.4,208.838242
2,1881.0,1114.8,398.534597
3,1854.0,1129.0,263.410811
4,1850.0,1171.6,389.641433
5,1877.0,1050.8,391.133704
6,1876.0,1151.2,358.957496
7,1874.0,1145.2,350.437578
8,1882.0,1114.4,353.375974
9,1844.0,1149.0,168.634600


In [204]:
print('\033[1m', instance_name , '\033[0m' , 'Shape' , df_result[0:15].shape)
print( ' Average  : \n' , df_result.mean())
print( '\n Maximum  : \n' , df_result.max())
print( '\n  Minimum : \n' , df_result.min())

 test_90k_20ta_10.txt  Shape (15, 3)
 Average  : 
 leader_obj      1872.000000
follower_obj    1134.800000
CPU_time         301.098285
dtype: float64

 Maximum  : 
 leader_obj      1902.000000
follower_obj    1190.200000
CPU_time         478.080916
dtype: float64

  Minimum : 
 leader_obj      1844.000000
follower_obj    1050.800000
CPU_time         165.311212
dtype: float64


In [895]:
## PRINTING THE SOLUTION   -- CPLEX   #####################
# Opt_Prices = sorted(Ref_Set1, key=lambda x: x[1], reverse=True )[0][0]
# Cust_Assign =  master_lower_level( K, J , Opt_Prices, R_K, BUDGETS, S_K   )
# print('Optimal Prices are ', Opt_Prices)
# print('Objective of Leader and Follower are',  Cust_Assign[2], Cust_Assign[1]  )
# print('Assignment of Customers ', prod_number( Cust_Assign[0], K , J ))

In [176]:
## Ensuring Feasibility #######################
soln = master_lower_level(K, J ,  Opt_Prices , R_K, BUDGETS, S_K  ) 

print('From Scatter Search', Cust_Assign[0] )

print('Cplex ', prod_number( soln[0], K , J )  )
if ( Cust_Assign[0] == prod_number( soln[0], K , J )):
    print('Optimal and Objective Value of leader and follower obtained from Cplex are', soln[2], soln[1]  )
else:
    print('Not Optimal')
    print('Not Optimal and Objective Value of leader and follower obtained from Recheck function are', soln[2], soln[1]  )


From Scatter Search [6, 2, 4, 3, 8, 4, 4, 4, 1, 10]
Cplex  [6, 2, 4, 3, 8, 4, 4, 4, 1, 10]
Optimal and Objective Value of leader and follower obtained from Cplex are 191.0 65.60000000000001


In [338]:
# Combination Calc 
# import operator as op
# from functools import reduce

# def ncr(n, r):
#     r = min(r, n-r)
#     numer = reduce(op.mul, range(n, n-r, -1), 1)
#     denom = reduce(op.mul, range(1, r+1), 1)
#     return numer // denom  # or / in Python 2

In [8]:
### STEP 1 -------------------------------------------------------------------
# ---------- Computing Lower Bound 
min_list = [max(BUDGETS)] * J

for i in range(0,J):
    for j in range(0,K):
        if min_list[i] > R_K[j][i]:
            min_list[i] = R_K[j][i]
            
lb = min_list

# -------------Computing Upper Bound 
max_list = [0] * J

for i in range(0,J):
    for j in range(0,K):
#         if max_list[i] < R_K[j][i]:
            max_list[i] = max(BUDGETS)

ub = max_list

In [9]:
def sub_range(lb, ub , K):
    range_width = 5
    no_of_subrange = math.ceil( ( max(ub)-max(lb) ) /range_width)
    subrange_matrix = [[0 for x in range(2)] for y in range(no_of_subrange)] 
#     print(no_of_subrange,subrange_matrix)
    val  = max(lb)
    for k in range(0, no_of_subrange):
        subrange_matrix[k][0] = val  + k*range_width
        subrange_matrix[k][1] = val + (k+1) * range_width
        
        if  subrange_matrix[k][1] > max(ub):
            subrange_matrix[k][1] = max(ub)
#     print(subrange_matrix)
    return no_of_subrange , subrange_matrix

In [10]:
def diversification_generation(K):
    no_of_subrange,  subrange_matrix =  sub_range(lb, ub , K)
    freq_matrix = [[0 for x in range(K)] for y in range(no_of_subrange )] 
    PRICE_DIVERSE_SET = []

    for i in range(0, P_Size):
        Nest_Price = []
        for k in range(K):
#             random.seed(12)
            sub_range_select = random.randint(0, no_of_subrange-1 )
            number_generate = random.randint( subrange_matrix[sub_range_select][0]  , subrange_matrix[sub_range_select][1])  
            freq_matrix[ sub_range_select][k] += 1
            Nest_Price.append(number_generate)
        PRICE_DIVERSE_SET.append( Nest_Price)
#     print(PRICE_DIVERSE_SET)
    
    return PRICE_DIVERSE_SET, freq_matrix
# PRICE_DIVERSE_SET, freq_matrix = diversification_generation(K   )

In [11]:
# PRICE_DIVERSE_SET

In [12]:
### STEP 2  Nelder and Mead Simplex CPLEX used -------------------------------------------------------------------
# def objective(  p):
#     obj = 0
#     i=0 
#     print('Value of Price received from scipy optimize',p)
#     cust_ass =  master_lower_level(K, J ,  p , R_K, BUDGETS, S_K  )
#     print('Product Selected ', prod_number(cust_ass[0], K, J  ) )
#     No_Prod = J
#     print('Objective from cplex', cust_ass[2])                                    
# # The function is working fine, as the objective value from cplex matches with the obj function of below code 
#     for value in cust_ass[0] :  
#         obj = obj +  p[i] * value 
#         i += 1
#         if (i==No_Prod):
#             i=0
            
#     print(obj)
#     return  -obj

In [13]:
#  Nelder and Mead Simplex Heuristic is Used  -----------------------------------------------------
def objective_h(  p):
    obj = 0
#     print('Value of Price received from scipy optimize',p)
    cust_ass_h = lower_level_optimal(K,J, p , BUDGETS , R_K, S_K ) 
    No_Prod = J
#     print('Cust Selection', cust_ass_h[0])
#     print('From Heuristic ', cust_ass_h[2]) 
# The function is working fine, as the objective value from heuristic matches with the obj function of below code 
    for index_p in cust_ass_h[0] : 
        obj = obj +  p[index_p-1]   
            
#     print(obj)
    return  -obj
# THE CPLEX VERSION AND THE HEURISTIC VERSION WORKS THE SAME 

In [14]:
def improvement_method(PRICE_SET_TO_IMPROVE):
    PRICE_IMPROVE_SET = []
    PRICE_IMPROVE_SET_R = []
    for i in range(0, len(PRICE_SET_TO_IMPROVE)  ):
        result = scipy.optimize.minimize(objective_h, PRICE_SET_TO_IMPROVE[i], method='nelder-mead', options={ 'maxiter': 5, 'disp': False }) 
        PRICE_IMPROVE_SET.append(result['x'].tolist())

#         print(PRICE_IMPROVE_SET )
    PRICE_IMPROVE_SET = price_roundoff(PRICE_IMPROVE_SET)
    
#     print('\n\n', PRICE_IMPROVE_SET )
#     print(type(result))
#     print(  result['x'] )
#     print('Status : %s' % result['message'])
#     print('Total Evaluations: %d' % result['nfev'])
    return PRICE_IMPROVE_SET

# PRICE_IMRPOVE_SET = improvement_method(PRICE_DIVERSE_SET)   
# 

In [15]:
# PRICE_IMRPOVE_SET

In [16]:
# ---------FUNCTION TO ROUND OFF PRICES FROM IMPROVEMENT OR COMBINED ----------
def price_roundoff(Set_of_Prices):
    for i in range(0 , len(Set_of_Prices)):
        index = 0
        for price in Set_of_Prices[i]:
            if type(price) != int:
#                 print(price)
                if (price.is_integer() != True) :
                    Set_of_Prices[i][index] = math.ceil(price)
                index +=1
            
    return Set_of_Prices

# price_roundoff(PRICE_IMRPOVE_SET)

In [17]:
# df_PR = pd.DataFrame(PRICE_IMRPOVE_SET )
# df_PR 

In [18]:
### STEP 3  High Quality -----------------------------------------------------
def high_quality_subset(PRICE_IMRPOVE_SET ):          # Generates the initial part of the ref_set --> high quality solutions
    P_Set =   [[0 for x in range(3)] for y in range( P_Size-b1 )] 
    HQ_Set =  [[0 for x in range(2)] for y in range(P_Size)] 
    for i in range(0,P_Size):
        HQ_Set[i][0] =  PRICE_IMRPOVE_SET[i]  
#         cust_ass_hqs  =  master_lower_level(K, J ,  PRICE_IMRPOVE_SET[i]  , R_K, BUDGETS, S_K  )   ## IF CPLEX USED 
        cust_ass_hqs  =  lower_level_optimal(K,J, PRICE_IMRPOVE_SET[i]  , BUDGETS , R_K, S_K )                 ## IF Heuristic USED 
        HQ_Set[i][1] =  cust_ass_hqs[2]  
        
    HQ_Sorted_Set = sorted(HQ_Set, key=lambda x: x[1], reverse=True)
    i = 0
    for elem in range(b1, P_Size  ):
        P_Set[i][0] = HQ_Sorted_Set[elem][0] 
        P_Set[i][1] = HQ_Sorted_Set[elem][1] 
        i += 1
    HQ_Sorted_Set = HQ_Sorted_Set[:b1]
    
#     for i in range( 0 , len(HQ_Sorted_Set)) :
#         HQ_Sorted_Set[i].pop(1)
        
    return HQ_Sorted_Set , P_Set

# Ref_Set1 , P_Set =  high_quality_subset(PRICE_IMRPOVE_SET )
# print(Ref_Set1)
# df_R1 = pd.DataFrame( Ref_Set1   )
# df_R1
# df_P = pd.DataFrame( P_Set   )
# df_P

In [19]:
# Starting out with HQ_Sorted_Set as Ref_Set by adding diverse elements it will eventually become the ref set
# Ref_Set = copy.deepcopy( Ref_Set1 )
#  NOTEE  Ref_Set = Ref_Set_Final 
# print(Ref_Set)

In [20]:
### Diverse Subset i.e. RefSet2-----------------------------------------------------
def diverse_subset(P_Set, Ref_Set  ):
    Ref_Set2 = []
    no_of_diverse_subset = b2 
    for no in range(0 , no_of_diverse_subset):  # How many diverse solutions needed ? 
        max_e = 0
        for p in range( 0 , len(P_Set) ):    #Calc eucl distance for each soln in P_Set 
#             print('Loop of element %d in P_set' %(p))   
            max_index = 0
            min_e_p = 1000
            for i in range( 0 ,len(Ref_Set) ):    #Calc eucl distance for each soln in P_Set for each soln in REF SET 
#                 print('Euclidean distance for %d with ref_set element %d' %(p,i) )
                eDistance = distance.euclidean( P_Set[p][0] , Ref_Set[i][0]  )  #looks like it's working fine for individual
#                 print(eDistance)
                if eDistance < min_e_p:   # Finding dmin for the particular soln of P_Set
                    min_e_p = eDistance
                    P_Set[p][2] = min_e_p
#             print('Minimum distance obtained for element %d in P_set is :%2.2f' %(p,min_e_p))
#             FInding the max d_min value 
            if min_e_p > max_e:
                max_e = min_e_p
                max_index = p

#         print('Max d_min value obtained is for element %d in P_set is :%2.2f' %(max_index,max_e))
#         print(P_Set[max_index])
        Ref_Set.append(P_Set[max_index] )
        Ref_Set2.append(P_Set[max_index] )
        P_Set.pop(max_index)

    return Ref_Set, Ref_Set2

# Ref_Set_Final, Ref_Set2 = diverse_subset(P_Set, Ref_Set  )

In [21]:
#  Visualizing P_Set
def P_Set_F(P_Set):
    P_Set_Nest = []
    P_Set_Edist = []
    for elem in P_Set:
    #     print(elem)
        P_Set_Nest.append(elem[0])
        P_Set_Edist.append(elem[2])

    products = ["Prod1","Prod2","Prod3","Prod4","Prod5","Prod6","Prod7","Prod8","Prod9","Prod10" ]
    df_P = pd.DataFrame(P_Set_Nest , columns = products)
    df_P['Edist'] = P_Set_Edist
    
    return df_P

# P_Set_F(P_Set)

In [22]:
#  Visualizing Ref_Set1
def Ref_Set1_F(Ref_Set1):
    Ref_Set_Nest = []
    Ref_Set_N_Obj = []
    for elem in Ref_Set1:
    #     print(elem)
        Ref_Set_Nest.append(elem[0])
        Ref_Set_N_Obj.append(elem[1])
  
    products = ["Prod1","Prod2","Prod3","Prod4","Prod5","Prod6","Prod7","Prod8","Prod9","Prod10" ]
    df_Ref1 = pd.DataFrame( Ref_Set_Nest , columns = products)
    df_Ref1['Obj'] = Ref_Set_N_Obj
    
 
    return df_Ref1
# Ref_Set1_F(Ref_Set1) 

In [23]:
#  Visualizing Ref_Set2
def Ref_Set2_F(Ref_Set2):
    Ref_Set_Nest = []
    Ref_Set_N_Edist = []
    for elem in Ref_Set2:
#         print(elem)
        Ref_Set_Nest.append(elem[0])
        Ref_Set_N_Edist.append(elem[2])
    products = ["Prod1","Prod2","Prod3","Prod4","Prod5","Prod6","Prod7","Prod8","Prod9","Prod10" ]
    df_Ref2 = pd.DataFrame( Ref_Set_Nest , columns = products)
    df_Ref2['Edist'] = Ref_Set_N_Edist
 
    return df_Ref2

# Ref_Set2_F(Ref_Set2)

In [24]:
### STEP 4 Subset Generation Method  -------------------------------------------------------------------
import itertools
iter_list = []
for i in range(0,b):
    iter_list.append(i)
no_of_subsets = itertools.combinations( iter_list, 2)
subsets = set(no_of_subsets)

In [25]:
# subsets

In [26]:
### STEP 5 Solution Combination Method  -------------------------------------------------------------------
def solution_combination(subset1,subset2):
    r = random.random()
#     print(r)
#     comb_soln =  [0 for x in range(3)]   
    news1 = []
    news2 = []
    news3 = []
    for k in range(0,len(subset1[0])): 
        d = r * ( subset2[0][k] - subset1[0][k] )/2 
        news1.append(subset1[0][k] - d)
        news2.append(subset1[0][k] + d)
#         news2.append(subset1[k] + d)
        news3.append(subset2[0][k] + d)
    
    comb_soln = [ news1, news2 , news3 ]
    
    return  comb_soln
#     print(news1 , news2 , news3)
# combined_solutions = solution_combination(Ref_Set_Final[0],Ref_Set_Final[1])

In [27]:
# Dmin calculator for combined solutions 
def dmin_combinedsol(combined_sol, Ref_Set_Final ):
#     print(combined_sol)
    # CHECKING IF new soln has better dmin value 
    min_e_p = 1000
    for i in range( 0 ,len(Ref_Set_Final) ):    #Calc eucl distance for combined solution for each soln in REF SET 
        max_index = 0
#         print('Euclidean distance for combined solnt with ref_set element %d' %(i) )
        eDistance = distance.euclidean( combined_sol , Ref_Set[i][0]  )  #looks like it's working fine for individual
#         print(eDistance)
        if eDistance < min_e_p:
            min_e_p = eDistance
#     print('Minimum distance obtained for combined soln   is :%2.2f' %( min_e_p))
    
    return min_e_p

In [28]:
# Worst dmin value in Ref_Set2 
def worst_dmin(Ref_Set2):
    Ref_Set2 = sorted(Ref_Set2, key=lambda x: x[2], reverse=False)
    worst_dmin_val = Ref_Set2[0][2]
#     return Ref_Set2

    return worst_dmin_val


In [29]:
# Worst Obj value in Ref_Set1 
def worst_objf(Ref_Set1):
    Ref_Set1 = sorted(Ref_Set1, key=lambda x: x[1], reverse=True)
    lower_level_obj_of_worst_in_refset =   lower_level_optimal(K,J, Ref_Set1[-1][0]  , BUDGETS , R_K, S_K )
    lower_level_obj = lower_level_obj_of_worst_in_refset[1]
    
    return Ref_Set1[-1][1] ,  lower_level_obj


In [30]:
# combined_solutions = solution_combination(Ref_Set_Final[0],Ref_Set_Final[1])
def combined_sol_unique_f(combined_sol, Ref_Set_Final):
    combined_sol_unique = True 
    #     print('Combined soln is \n', combined_sol )
    for set_element in Ref_Set_Final: 
        if set_element[0] == combined_sol:
#                 print('Equal found , Break Happened ')
            combined_sol_unique = False  
            break 
        
    return combined_sol_unique

In [198]:
########### READ DATA #####################
def read_instance():
    os.chdir("D:\AA MASC\C Codes\FINAL\Big_Instance")
    with open("test_90k_20ta_10.txt", 'r' ) as file:         # Hardcoded file name 
        lines = file.readlines()
        text_file = file.read()
        file.close()
        
    i=1
    TF = []
    Rdash = []
    RR = []

    for line in lines:
        if i==1:
            J = int (line.split()[2])      # No. of Products 
            i+=1
#             print(J)
            continue

        if i==2:
            K = int (line.split()[2] )     # No. of Customers 
            i+=1
#             print(K)
            continue

        if i==3:
            i+=1
            continue 

        if i == 4:                       # i = 4 , denotes 4th line 
    #         print(line)
            for iter in range(0,K):
#                 print(iter)
#                 print(line.split()[iter+3 ])
                TF.append( int ( line.split()[ iter+3 ] ) ) 
#                 print(TF)
            i+=1
            continue

        if ( 4 < i < 5+ J):
            for iter in range(0,K):
#                 print('iter' , iter)
                if i == 5:
#                     print(line.split()[iter+3])
                    Rdash.append( int ( line.split()[ iter+3  ] ) ) 
                else:
#                     print(line.split()[iter])
                    Rdash.append( int ( line.split()[ iter  ] ) ) 
            i+=1
            RR.append(Rdash)
            Rdash=[]        
            continue
        
#         print(RR)
        NestList = []
        R_K =[] 
        BUDGETS = []
        
        for k in range(0,K):                   # For each customer 
            for j in range(0,J):   
#                 print( TF[j])
                NestList.append( TF[k] -  RR[j][k]) 
            R_K.append(NestList)
            BUDGETS.append(max(NestList))
            NestList=[]
        
        
         
        return K,J , R_K, BUDGETS

In [ ]:
instance_name = 'test_90k_20ta_10.txt'
K,J , R_K, BUDGETS = read_instance()

#  Create a blank dataframe , every time I read a new instance  
cols=["leader_obj",'follower_obj','CPU_time']
df_result  =pd.DataFrame(  columns = cols  )

In [ ]:
S_K = [[0, 15, 16, 18, 0, 0, 9, 0, 14, 6, 17, 8, 19, 7, 0, 10, 13, 11, 20, 12], [12, 15, 13, 19, 8, 0, 7, 11, 14, 16, 18, 17, 0, 0, 20, 0, 9, 0, 10, 6], [0, 11, 18, 0, 7, 16, 14, 12, 8, 10, 13, 15, 0, 0, 17, 0, 6, 19, 9, 20], [0, 0, 11, 9, 17, 20, 16, 10, 19, 0, 7, 15, 13, 18, 0, 6, 12, 0, 8, 14], [16, 8, 0, 12, 14, 13, 7, 18, 11, 17, 0, 9, 6, 0, 20, 10, 0, 0, 19, 15], [8, 0, 19, 12, 7, 18, 9, 0, 15, 20, 16, 0, 13, 6, 14, 17, 0, 0, 11, 10], [0, 0, 14, 18, 7, 10, 19, 15, 13, 17, 6, 0, 12, 16, 11, 0, 8, 20, 9, 0], [12, 0, 11, 9, 17, 19, 18, 14, 6, 15, 16, 20, 0, 0, 0, 0, 8, 7, 13, 10], [16, 13, 0, 19, 7, 15, 8, 20, 0, 0, 9, 6, 18, 11, 0, 17, 0, 10, 12, 14], [7, 12, 14, 0, 19, 15, 16, 17, 0, 0, 10, 6, 0, 20, 11, 0, 13, 18, 9, 8], [10, 0, 6, 11, 17, 0, 20, 12, 14, 15, 7, 19, 0, 18, 9, 0, 0, 8, 13, 16], [18, 14, 0, 0, 6, 12, 11, 17, 10, 19, 9, 7, 0, 8, 13, 0, 16, 20, 0, 15], [15, 20, 17, 13, 7, 0, 0, 0, 0, 12, 11, 8, 0, 6, 9, 16, 14, 10, 19, 18], [16, 0, 9, 11, 0, 8, 18, 14, 10, 20, 0, 7, 13, 15, 6, 19, 12, 0, 17, 0], [0, 9, 0, 6, 16, 7, 0, 15, 11, 17, 0, 14, 20, 12, 10, 18, 0, 8, 19, 13], [0, 16, 0, 10, 0, 20, 18, 13, 17, 6, 11, 15, 12, 7, 8, 0, 9, 19, 0, 14], [10, 13, 12, 9, 6, 20, 18, 16, 0, 17, 0, 0, 8, 14, 0, 15, 11, 0, 7, 19], [13, 19, 9, 14, 11, 8, 0, 0, 6, 18, 12, 10, 0, 15, 0, 17, 16, 0, 20, 7], [16, 15, 12, 14, 0, 17, 19, 7, 11, 0, 18, 6, 9, 8, 20, 13, 0, 0, 0, 10], [12, 17, 8, 13, 0, 0, 0, 14, 6, 18, 0, 9, 19, 20, 0, 10, 11, 15, 16, 7], [0, 0, 14, 16, 0, 10, 13, 15, 17, 9, 7, 18, 19, 6, 8, 11, 0, 20, 0, 12], [16, 19, 11, 18, 12, 14, 0, 10, 7, 0, 8, 20, 6, 13, 17, 9, 0, 0, 0, 15], [19, 16, 8, 10, 6, 18, 13, 0, 9, 11, 0, 0, 20, 14, 17, 12, 15, 7, 0, 0], [0, 13, 10, 20, 9, 12, 18, 16, 0, 0, 19, 14, 15, 7, 0, 11, 0, 8, 6, 17], [0, 20, 14, 0, 9, 18, 7, 13, 19, 0, 0, 16, 8, 10, 11, 12, 0, 6, 15, 17], [0, 7, 20, 14, 0, 18, 13, 19, 0, 6, 12, 17, 0, 8, 10, 15, 16, 0, 9, 11], [0, 11, 12, 0, 0, 16, 0, 17, 14, 6, 9, 20, 10, 15, 7, 13, 19, 18, 8, 0], [6, 0, 0, 0, 10, 9, 11, 12, 7, 0, 18, 13, 17, 14, 20, 8, 19, 15, 16, 0], [15, 11, 0, 17, 16, 13, 0, 7, 0, 0, 6, 0, 10, 20, 8, 14, 12, 18, 19, 9], [0, 14, 6, 13, 17, 9, 20, 11, 15, 0, 18, 7, 0, 16, 19, 8, 10, 12, 0, 0], [12, 13, 14, 11, 17, 0, 9, 0, 16, 18, 15, 0, 0, 7, 19, 8, 0, 20, 10, 6], [0, 17, 19, 0, 20, 16, 15, 7, 0, 12, 0, 8, 14, 13, 11, 10, 6, 0, 9, 18], [13, 8, 19, 0, 11, 0, 0, 0, 6, 18, 12, 9, 20, 10, 17, 7, 15, 14, 0, 16], [17, 10, 8, 14, 20, 11, 18, 7, 15, 16, 9, 0, 12, 0, 0, 19, 0, 13, 0, 6], [14, 7, 0, 18, 12, 19, 13, 11, 15, 0, 16, 0, 0, 6, 10, 0, 8, 9, 17, 20], [20, 8, 13, 0, 19, 11, 14, 16, 0, 12, 0, 18, 0, 17, 0, 6, 7, 9, 10, 15], [0, 18, 11, 12, 0, 7, 15, 10, 8, 16, 17, 0, 20, 19, 14, 13, 6, 0, 0, 9], [6, 10, 0, 0, 16, 18, 0, 11, 0, 0, 19, 12, 20, 9, 13, 8, 7, 14, 15, 17], [9, 15, 16, 11, 6, 14, 12, 19, 0, 10, 7, 17, 13, 0, 8, 0, 0, 0, 18, 20], [18, 13, 0, 0, 20, 6, 10, 14, 0, 0, 16, 15, 9, 0, 12, 17, 19, 11, 8, 7], [0, 20, 6, 14, 0, 8, 12, 15, 13, 16, 18, 0, 19, 11, 7, 10, 17, 0, 9, 0], [18, 11, 0, 0, 17, 0, 12, 19, 8, 7, 15, 6, 16, 0, 14, 20, 0, 9, 10, 13], [0, 19, 20, 17, 9, 14, 16, 15, 10, 0, 0, 13, 7, 0, 6, 8, 0, 18, 11, 12], [17, 19, 18, 0, 6, 11, 7, 0, 0, 8, 0, 13, 10, 20, 0, 9, 14, 15, 16, 12], [10, 8, 7, 11, 6, 14, 12, 0, 18, 15, 0, 17, 20, 9, 0, 13, 0, 19, 16, 0], [19, 14, 12, 0, 0, 6, 17, 11, 0, 20, 0, 15, 16, 7, 10, 13, 18, 0, 8, 9], [11, 17, 20, 7, 0, 19, 18, 13, 16, 0, 0, 0, 10, 0, 9, 8, 6, 14, 15, 12], [9, 19, 0, 11, 0, 17, 18, 20, 16, 0, 6, 15, 10, 14, 13, 12, 8, 0, 0, 7], [15, 20, 0, 8, 0, 9, 18, 14, 13, 12, 7, 0, 6, 0, 10, 16, 19, 17, 0, 11], [11, 17, 16, 19, 13, 0, 15, 9, 0, 7, 8, 0, 6, 0, 14, 10, 0, 12, 20, 18], [18, 19, 7, 11, 14, 0, 8, 10, 6, 12, 0, 9, 0, 20, 15, 0, 16, 0, 17, 13], [0, 20, 0, 9, 10, 15, 16, 14, 11, 0, 8, 18, 7, 17, 6, 19, 13, 0, 12, 0], [0, 18, 15, 0, 11, 9, 13, 0, 20, 12, 7, 19, 8, 14, 16, 17, 10, 6, 0, 0], [0, 9, 10, 17, 7, 19, 0, 8, 0, 13, 11, 0, 0, 20, 14, 15, 6, 16, 12, 18], [12, 0, 19, 0, 16, 0, 13, 17, 7, 18, 0, 6, 10, 9, 0, 8, 14, 20, 15, 11], [0, 12, 11, 19, 18, 0, 0, 6, 14, 0, 20, 15, 7, 17, 9, 10, 8, 13, 16, 0], [16, 14, 7, 17, 0, 15, 0, 8, 13, 0, 11, 19, 9, 0, 20, 10, 6, 0, 12, 18], [11, 13, 0, 15, 10, 0, 19, 8, 0, 7, 12, 20, 16, 9, 17, 6, 0, 0, 14, 18], [6, 16, 9, 13, 0, 7, 19, 0, 20, 0, 10, 0, 14, 17, 0, 8, 15, 12, 18, 11], [0, 19, 14, 0, 16, 17, 0, 7, 15, 12, 20, 0, 9, 8, 11, 0, 18, 13, 10, 6], [0, 15, 12, 0, 20, 17, 8, 0, 18, 6, 9, 11, 0, 19, 10, 7, 14, 0, 16, 13], [8, 19, 0, 16, 13, 14, 0, 11, 20, 12, 15, 10, 0, 18, 7, 0, 6, 0, 17, 9], [16, 13, 0, 9, 18, 20, 17, 15, 0, 10, 6, 11, 14, 19, 0, 0, 12, 0, 8, 7], [17, 11, 9, 13, 19, 18, 0, 14, 0, 16, 0, 12, 20, 8, 7, 0, 10, 15, 6, 0], [9, 0, 6, 13, 0, 18, 20, 16, 8, 12, 10, 0, 15, 0, 17, 19, 0, 14, 7, 11], [20, 19, 8, 16, 11, 9, 7, 17, 0, 15, 10, 6, 0, 14, 0, 0, 18, 12, 0, 13], [6, 0, 18, 9, 14, 0, 19, 13, 0, 10, 8, 11, 12, 7, 17, 16, 15, 0, 0, 20], [0, 14, 0, 13, 16, 0, 8, 10, 6, 0, 0, 20, 7, 9, 11, 18, 12, 15, 17, 19], [0, 13, 0, 20, 14, 10, 16, 15, 11, 18, 0, 0, 7, 19, 0, 17, 9, 12, 8, 6], [0, 0, 17, 13, 10, 9, 19, 12, 8, 6, 0, 15, 0, 0, 11, 14, 18, 20, 7, 16], [9, 10, 7, 17, 18, 8, 0, 6, 11, 16, 19, 14, 12, 0, 0, 15, 0, 13, 0, 20], [0, 14, 8, 13, 17, 9, 20, 0, 19, 16, 15, 7, 11, 6, 12, 0, 0, 0, 10, 18], [8, 15, 16, 14, 0, 10, 19, 7, 0, 9, 0, 12, 13, 17, 18, 11, 0, 0, 20, 6], [20, 18, 0, 0, 19, 0, 14, 11, 9, 15, 6, 0, 12, 8, 17, 16, 10, 13, 7, 0], [0, 19, 9, 13, 7, 8, 18, 0, 0, 15, 0, 6, 14, 0, 17, 20, 12, 16, 11, 10], [11, 0, 15, 0, 8, 17, 0, 10, 14, 16, 0, 20, 19, 0, 7, 18, 12, 6, 9, 13], [9, 17, 10, 15, 11, 20, 13, 0, 0, 6, 0, 16, 8, 12, 0, 0, 18, 7, 14, 19], [13, 18, 0, 17, 9, 7, 0, 19, 6, 11, 20, 8, 0, 0, 15, 0, 10, 12, 14, 16], [16, 0, 18, 0, 17, 8, 19, 0, 15, 20, 0, 7, 11, 14, 13, 6, 12, 10, 0, 9], [0, 18, 14, 20, 0, 9, 19, 0, 8, 16, 12, 15, 17, 6, 7, 0, 13, 10, 0, 11], [14, 6, 9, 0, 7, 10, 0, 20, 19, 18, 0, 8, 17, 15, 16, 12, 11, 0, 0, 13], [13, 14, 10, 17, 6, 15, 9, 0, 8, 0, 7, 19, 11, 0, 0, 12, 18, 0, 16, 20], [15, 16, 7, 9, 19, 0, 0, 14, 0, 8, 0, 6, 0, 20, 12, 18, 13, 10, 17, 11], [7, 19, 6, 9, 15, 0, 17, 0, 18, 20, 0, 16, 10, 12, 11, 13, 0, 0, 8, 14], [16, 0, 11, 13, 0, 8, 10, 12, 0, 20, 15, 0, 7, 18, 17, 14, 19, 9, 6, 0], [20, 10, 0, 8, 9, 7, 14, 15, 17, 0, 0, 0, 18, 6, 12, 13, 19, 0, 11, 16], [11, 17, 6, 0, 0, 0, 0, 20, 14, 18, 10, 13, 8, 16, 12, 19, 15, 0, 7, 9], [0, 6, 13, 19, 9, 12, 20, 8, 17, 15, 16, 11, 0, 14, 0, 18, 0, 0, 10, 7], [0, 12, 14, 11, 17, 0, 15, 0, 10, 18, 0, 20, 16, 9, 13, 7, 6, 19, 0, 8], [0, 11, 7, 20, 12, 17, 0, 6, 0, 13, 16, 0, 10, 15, 8, 19, 0, 14, 18, 9]]


In [31]:
# -------------CPLEX LOWER LEVEL ------------------
def master_lower_level( K, J , PRICES, R_K, BUDGETS, S_K  , **kwargs):  
   
     
    #iterations +=1 
    #     ----- Result Variable (Variables to store) ----------------
    Cust_Ass =[]
    lower_objective = 0
    
    mdl = Model(name='lowerlevel' )

    weight =0.8          # Weight 
   
    R = range(0,K)
    # ------ decision variables ---
    x  = mdl.binary_var_matrix(K,J ,"cust_assign", key_format ="_%s")
     
    # --------constraints 1---
    for i in R:  
        mdl.add_constraint( mdl.sum(  x[i,j] for j in range(0,J) )  <= 1   )

    # --------- constraints2 ---
    for i in R:
        mdl.add_constraint( mdl.sum(   x[i,j]* PRICES[j] for j in range(0,J)  )  <= BUDGETS[ i]    )
   
    # --------- constraints3 ---
    for k in R:
        for j in range(0,J):
            mdl.add_constraint(    x[k,j] <=   S_K[k][j]  )
  
    # ------------ objective ---         
    mdl.maximize( mdl.sum(   weight*  x[i,j ] *S_K[i][j] +  x[i,j ] *(1-weight)* (R_K[i][j] - PRICES[j]  )     for i in R for j in range(0,J)) )      
    
    # ------------ KPI  ---   
    mdl.add_kpi(  mdl.sum( x[i,j ]*PRICES[j]      for i in R for j in range(0,J) )  , publish_name = 'UpperObjective')  

 
    msol = mdl.solve()                 #log_output=True              # msol is a data structure of the class Solve Solution
    # ----- Retrieving values of decision variables 
    for i in range(0,K):
            for j in range(0,J):
                    ld = msol.get_value(x[i,j ])
                    Cust_Ass.append(ld) 
                    
    lower_objective = msol.get_objective_value()                # Retrieves obj value 
    upper_objective = msol.kpi_value_by_name('UpperObjective')
    
    return Cust_Ass , lower_objective ,upper_objective

In [33]:
# FUNCTION TO SOLVE LOWER LEVEL TO OPTIMALITY 
def lower_level_optimal(K,J, PRICES , BUDGETS , R_K, S_K ):
    weight = 0.8
    Best_Cust_Ass = []
    lower_obj = 0
    upper_obj = 0
    for i in range(K):         # For each customer 
#         print('Customer ', i)
        opt_val = 0
        opt_sel= 0
        prod_selection_flag = 0
        for j in range(J):
            if ( S_K[i][j] != 0 and BUDGETS[i] >= PRICES[j] ):
#                 print( 'Product ', j,  BUDGETS[i] , PRICES[j])
                obj = weight*S_K[i][j] + (1-weight)* ( R_K[i][j] - PRICES[j])
                if obj >= opt_val:          # Checks whether the current obj is greater than equal to the previous best
                    if obj == opt_val:    # If current obj is = to previous best 
                        if ( PRICES[j] > PRICES[opt_sel] ):   # Checking if PRICE of current obj PRODUCT is > than price of previous best
                        # IF PRICE more than replace as it contributes to leader's objective better 
                            opt_val = obj
                            opt_sel = j+1
                            prod_selection_flag = 1

                    else : # (automatically satisfied) else strictly greater than simply replace 
                        opt_val = obj
                        opt_sel = j+1
                        prod_selection_flag = 1
#             print('Customer %d optimal selection is'   %(i+1), opt_sel ,prod_selection_flag )
        Best_Cust_Ass.append(opt_sel)
        lower_obj = lower_obj + opt_val 
        upper_obj = upper_obj + PRICES[opt_sel-1] * prod_selection_flag
        
    return Best_Cust_Ass, lower_obj , upper_obj

In [34]:
# ----------Funtion to Compute , Assignment 
def prod_number(cust_list, no_cust, no_prod  ):
    numbered_list = []
    iterator = 0
    for i in range(no_cust):
        flag = 0
        j = 0
        for j in range(no_prod ):  
#             print(int(str(i) + str(j)))
            if ( cust_list[iterator]  >  0 ) :  
                numbered_list.append(j+1)
                flag=1
            j += 1
            iterator += 1
        if flag ==0:
            numbered_list.append(0)
            
    return numbered_list

# This list contains actual index i.e. Product1, Prod 2 NOT python indices from 0,
# A value of 0 indicates customer did not made any purchase 

In [197]:
## Implementing Nelder method 
# https://docs.scipy.org/doc/scipy/reference/optimize.minimize-neldermead.html
# https://machinelearningmastery.com/how-to-use-nelder-mead-optimization-in-python/

In [ ]:
# SOLUTIONS 
P1  = [ ]
P2 = [24, 31, 23, 21, 24, 21, 20, 22, 19, 20]
P3  = [24.0, 27.0, 22.0, 24, 20.0, 13.0, 25.0, 27.0, 9, 20]
P4 = [26, 29, 24, 28, 26, 26, 23, 15, 21, 24]
P5 = 

In [38]:
# --------Visualizing and Comparing the Result ---------

def obj_evaluation(K,J, PRICES, BUDGETS, R_K, S_K):
    Obj_list = []
    weight = 0.8
    for i in range(K):         # For each customer 
#         print("Customer is %d"%(i+1))
        Nest_O = []
        for j in range(J):
#             print("Product is %d"%(j+1))
#             print("Reservation: %d and Rank: %d"%( R_K[i][j], S_K[i][j]))                   or BUDGETS[i] < PRICES[j]
            if (S_K[i][j] == 0 or PRICES[j]>BUDGETS[i] ):
#                 print("Customer %d , is unable to purchase Prod:%d as Budget is %d and Price is %d"%( i+1,j+1, BUDGETS[i], PRICES[j] ))
                obj = -1 
            else:
#                 obj = round( weight*S_K[i][j] + (1-weight)* ( R_K[i][j] - PRICES[j]), 2)  
                obj = weight*S_K[i][j] + (1-weight)* ( R_K[i][j] - PRICES[j])      
            Nest_O.append(obj)
        Obj_list.append(Nest_O)        
    return Obj_list

new_list = obj_evaluation(K,J, Opt_Prices ,BUDGETS,  R_K, S_K)

columns=["Cust1", "Cust2","Cust3","Cust4","Cust5","Cust6","Cust7", "Cust8", "Cust9", "Cust10"]
products = ["Prod1","Prod2","Prod3","Prod4","Prod5","Prod6","Prod7","Prod8","Prod9","Prod10" ]
df =pd.DataFrame(new_list, columns=products, index= columns   )
df['Budgets'] = BUDGETS
df['Selection'] =  Cust_Assign[1]
# df['Selection'] = prod_number( Cust_Assign[0] , K , J )
pric =[]
pric = copy.deepcopy(Opt_Prices)
pric.append(0)
pric.append(0)
df.loc['Prices'] = pric

df_RK = pd.DataFrame(R_K, columns=products, index= columns  )
# df_RK['Budgets'] = BUDGETS
df_SK = pd.DataFrame(S_K, columns=products ,index= columns )
df_concat = pd.concat([df_RK,df_SK] , axis = 1)

df

,Prod1,Prod2,Prod3,Prod4,Prod5,Prod6,Prod7,Prod8,Prod9,Prod10,Budgets,Selection
Cust1,6.8,6.4,-1.0,6.8,4.6,3.2,4.2,-1.0,2.4,1.4,29.0,62.8
Cust2,-1.0,7.2,-1.0,4.6,4.8,-1.0,-1.0,3.8,-1.0,3.2,22.0,62.8
Cust3,-1.0,6.8,-1.0,-1.0,6.0,-1.0,-1.0,2.2,-1.0,1.2,22.0,62.8
Cust4,-1.0,-1.0,-1.0,-1.0,6.4,-1.0,-1.0,-1.0,6.4,-1.0,18.0,62.8
Cust5,-1.0,-1.0,-1.0,-1.0,5.6,-1.0,-1.0,-1.0,-1.0,-1.0,15.0,62.8
Cust6,-1.0,-1.0,-1.0,5.8,6.4,-1.0,-1.0,-1.0,-1.0,-1.0,19.0,62.8
Cust7,-1.0,-1.0,-1.0,5.6,5.0,-1.0,-1.0,-1.0,-1.0,-1.0,18.0,62.8
Cust8,-1.0,-1.0,-1.0,6.4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,18.0,62.8
Cust9,-1.0,6.4,-1.0,-1.0,-1.0,-1.0,-1.0,3.4,4.4,1.8,23.0,62.8
Cust10,-1.0,-1.0,-1.0,5.0,-1.0,-1.0,-1.0,-1.0,5.2,-1.0,20.0,62.8


In [ ]:
# STEP BY STEP IMPLEMENTATION #####################

In [31]:
PRICE_DIVERSE_SET, freq_matrix = diversification_generation(K)
# PRICE_IMRPOVE_SET   = PRICE_DIVERSE_SET
PRICE_IMRPOVE_SET = improvement_method(PRICE_DIVERSE_SET) 

In [32]:
# PRICE_IMRPOVE_SET

In [33]:
Ref_Set1 , P_Set =  high_quality_subset(PRICE_IMRPOVE_SET )         #Storing the References Sets 
Ref_Set = copy.deepcopy( Ref_Set1 )
Ref_Set_Final, Ref_Set2 = diverse_subset(P_Set, Ref_Set  ) 

In [34]:
i = 1 
for subset in subsets:                                        # Combining solutions for each subset in the Ref Set
    combined_solutions = solution_combination( Ref_Set_Final[subset[0]], Ref_Set_Final[subset[1]])  #Gives 3 comb solns
#     print('combined solutions ',combined_solutions )
    imp_combined_solutions   = improvement_method(combined_solutions)
#     print('Improved  solutions \n ',i, imp_combined_solutions )
    i+=1 

In [38]:
Best_Prices = sorted(Ref_Set1, key=lambda x: x[1], reverse=True )[0][0]
Cust_Assign =  master_lower_level( K, J , Best_Prices, R_K, BUDGETS, S_K   )
print('Optimal Prices are ', Best_Prices)
print('Objective of Leader and Follower are',  Cust_Assign[2], Cust_Assign[1]  )
print('Assignment of Customers ', prod_number( Cust_Assign[0], K , J ))

Optimal Prices are  [19, 22, 21, 23, 28, 17, 16, 17, 18, 15]
Objective of Leader and Follower are 187.0 59.800000000000004
Assignment of Customers  [1, 2, 2, 9, 10, 1, 6, 7, 3, 9]


In [39]:
Heuristic_assign = lower_level_optimal(K,J, Best_Prices , BUDGETS , R_K, S_K )
print('Optimal Prices are ', Best_Prices)
print('Objective of Leader and Follower are',  Heuristic_assign[2], Heuristic_assign[1]  )
print('Assignment of Customers ', Heuristic_assign[0] )

Optimal Prices are  [19, 22, 21, 23, 28, 17, 16, 17, 18, 15]
Objective of Leader and Follower are 187 59.800000000000004
Assignment of Customers  [1, 2, 2, 9, 10, 1, 6, 7, 3, 9]
